2019/2020 - MDS-1 - Extracción, Transformación y Carga - Etl

Raquel Fort Serra

# Resolucion en SQL LogNasa

##### 1. Parsear el dataset para su tratamiento 

In [40]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [41]:
data = "apache.access.log_small" 
data = sc.textFile(data)

In [42]:
data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

In [43]:
import re

In [44]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)

In [45]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0]) #map con el segundo parseador, filtramos cuales de ellos han sido uno (que el paseador los pasa adecuadamente) y despues mapeamos
parsed_def = parsed_rdd.map(lambda line: map_log(line))

##### 2. Transformar en un sqldataframe

In [46]:
from pyspark.sql import SQLContext, Row 
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [47]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)

In [48]:
sql_data = parsed_def.map(lambda p: 
                         Row(Host = p[0], 
                             date = p[3][:11],
                            endpoint = p[6], code = p[-2],
                            size = p[-1]))

In [49]:
sql_data.take(1)

[Row(Host='in24.inetnebr.com', code='200', date='01/Aug/1995', endpoint='/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', size='1839')]

In [50]:
df = sqlContext.createDataFrame(sql_data)
df.registerTempTable("nasa")

### 3. Usando el RDD access_logs calcular:
- Mínimo, máximo y media del tamaño de las peticiones (size)
- Nº peticiones de cada código de respuesta (response_code)
- Mostrar 20 hots que han sido visitados más de 10 veces
- Mostrar los 10 endpoints más visitados
- Mostrar los 10 endpoints más visitados que no tienen código de respuesta = 200
- Calcular el nº de hosts distintos
- Contar el nº de hosts únicos cada día 
- Calcular la media de peticiones diarias por host
- Mostrar una lista de 40 endpoints distintos que generan código de respuesta = 404
- Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan 
- El top 5 de días que se generaron código de respuestas 404

#### Mínimo, máximo y media del tamaño de las peticiones (size) 

In [72]:
datos = sqlContext.sql("""
    SELECT * 
    FROM nasa 
    LIMIT 5
""")
datos.show()

+-----------------+----+-----------+--------------------+----+
|             Host|code|       date|            endpoint|size|
+-----------------+----+-----------+--------------------+----+
|in24.inetnebr.com| 200|01/Aug/1995|/shuttle/missions...|1839|
|  uplherc.upl.com| 304|01/Aug/1995|                   /|   0|
|  uplherc.upl.com| 304|01/Aug/1995|/images/ksclogo-m...|   0|
|  uplherc.upl.com| 304|01/Aug/1995|/images/MOSAIC-lo...|   0|
|  uplherc.upl.com| 304|01/Aug/1995|/images/USA-logos...|   0|
+-----------------+----+-----------+--------------------+----+



In [52]:
tamaño = sqlContext.sql("""
    SELECT MIN(size) AS Minimo, MAX(size) AS Maximo, ROUND(AVG(size), 2) AS Media 
    FROM nasa 
""")
tamaño.show()

+------+------+-------+
|Minimo|Maximo|  Media|
+------+------+-------+
|     -| 99942|16183.9|
+------+------+-------+



#### Nº peticiones de cada código de respuesta (response_code)

In [53]:
peticiones = sqlContext.sql("""
    SELECT SUM(size) AS Peticiones, code AS Codigo 
    FROM nasa 
    GROUP BY code
""")
peticiones.show()

+-----------+------+
| Peticiones|Codigo|
+-----------+------+
|5.5085536E7|   200|
|     4460.0|   302|
|       null|   404|
|       null|   403|
|        0.0|   304|
+-----------+------+



#### Mostrar 20 hots que han sido visitados más de 10 veces

In [65]:
hosts = sqlContext.sql("""
    SELECT COUNT(host), host
    FROM nasa 
    GROUP BY host
    ORDER BY COUNT(host) DESC
    LIMIT 20 
""")
hosts.show()

+-----------+--------------------+
|count(host)|                host|
+-----------+--------------------+
|         78|ix-min1-02.ix.net...|
|         71|     uplherc.upl.com|
|         59|port26.ts1.msstat...|
|         56|   h96-158.ccnet.com|
|         55|   in24.inetnebr.com|
|         54|thing1.cchem.berk...|
|         54|piweba3y.prodigy.com|
|         44|   adam.tower.com.au|
|         43|ip55.van2.pacifie...|
|         41|ppp1016.po.iijnet...|
|         40|hsccs_gatorbox07....|
|         40|www-b2.proxy.aol.com|
|         39|www-d1.proxy.aol.com|
|         37|        133.43.96.45|
|         37|  port13.wavenet.com|
|         33|       pc-heh.icl.dk|
|         32|haraway.ucet.ufl.edu|
|         31|       193.84.66.147|
|         30|www-c1.proxy.aol.com|
|         29|term1-24.sb.west.net|
+-----------+--------------------+



#### Mostrar los 10 endpoints más visitados

In [60]:
endpoints =sqlContext.sql("""
    SELECT COUNT(endpoint), Endpoint
    FROM nasa 
    GROUP BY endpoint 
    ORDER BY COUNT(endpoint) DESC
    LIMIT 10
""")
endpoints.show()

+---------------+--------------------+
|count(endpoint)|            Endpoint|
+---------------+--------------------+
|            167|/images/KSC-logos...|
|            160|/images/NASA-logo...|
|            122|/images/MOSAIC-lo...|
|            120|/images/WORLD-log...|
|            118|/images/USA-logos...|
|            106|/images/ksclogo-m...|
|             85|                   /|
|             74|/history/apollo/i...|
|             69|/images/launch-lo...|
|             66|/images/ksclogosm...|
+---------------+--------------------+



#### Mostrar los 10 endpoints más visitados que no tienen código de respuesta = 200

In [67]:
endpoints2 = sqlContext.sql("""
    SELECT COUNT(endpoint), Endpoint
    FROM nasa
    WHERE code != 200
    GROUP BY endpoint 
    ORDER BY COUNT(endpoint) DESC
    LIMIT 10
""")
endpoints2.show()

+---------------+--------------------+
|count(endpoint)|            Endpoint|
+---------------+--------------------+
|             25|/images/NASA-logo...|
|             24|/images/KSC-logos...|
|             17|/images/MOSAIC-lo...|
|             17|/images/WORLD-log...|
|             16|/images/USA-logos...|
|             10|/images/ksclogo-m...|
|              8|/software/winvn/b...|
|              8|/images/construct...|
|              8|/software/winvn/w...|
|              6|/software/winvn/w...|
+---------------+--------------------+



#### Calcular el nº de hosts distintos   

In [79]:
hotsdistint = sqlContext.sql("""
    SELECT COUNT(DISTINCT host) AS HostsDistintos 
    FROM nasa
""")
hotsdistint.show()

+--------------+
|HostsDistintos|
+--------------+
|           311|
+--------------+



#### Contar el nº de hosts únicos cada día 

In [82]:
hostsdia = sqlContext.sql("""
    SELECT COUNT(DISTINCT host) AS HostsUnicos, date AS dia
    FROM nasa
    GROUP BY date
""")
hostsdia.show()

+-----------+-----------+
|HostsUnicos|        dia|
+-----------+-----------+
|        311|01/Aug/1995|
+-----------+-----------+



#### Calcular la media de peticiones diarias por host 

In [83]:
mediahost = sqlContext.sql("""
    SELECT date AS Fecha,  ROUND(AVG(size), 2) AS media, host
    FROM nasa
    GROUP BY date, host
""")
mediahost.show()

+-----------+--------+--------------------+
|      Fecha|   media|                host|
+-----------+--------+--------------------+
|01/Aug/1995| 20991.1|www-a2.proxy.aol.com|
|01/Aug/1995| 6676.71|celebrian.otago.a...|
|01/Aug/1995|  5774.4|sprite131.azstarn...|
|01/Aug/1995|  7280.0|  van10275.direct.ca|
|01/Aug/1995| 8274.28|   h96-158.ccnet.com|
|01/Aug/1995| 27739.0|engei.engei-hs.oy...|
|01/Aug/1995| 9324.53|      139.230.35.135|
|01/Aug/1995|  4574.5|thunderbox.ho.bom...|
|01/Aug/1995| 4173.63|        pm00.ldl.net|
|01/Aug/1995|31753.09|  cs1-08.leh.ptd.net|
|01/Aug/1995|15684.78|      198.248.59.123|
|01/Aug/1995| 5461.88|stockyard17.onram...|
|01/Aug/1995|  8041.0|dd01-045.compuser...|
|01/Aug/1995|   995.0|ip-pdx8-00.telepo...|
|01/Aug/1995|  3381.0|        203.10.76.34|
|01/Aug/1995| 9136.63|       168.78.14.166|
|01/Aug/1995|21918.29|ix-ir8-22.ix.netc...|
|01/Aug/1995|67891.67|ad11-013.compuser...|
|01/Aug/1995|34807.33|castles10.castles...|
|01/Aug/1995| 5048.38|  vcc7.lan

#### Mostrar una lista de 40 endpoints distintos que generan código de respuesta = 404 

In [89]:
endpoints404 = sqlContext.sql("""
    SELECT DISTINCT endpoint
    FROM nasa
    WHERE code = '404'
    LIMIT 40
""")
endpoints404.show()

+--------------------+
|            endpoint|
+--------------------+
|/pub/winvn/readme...|
|/elv/DELTA/uncons...|
|/history/apollo/a...|
|/history/apollo/a...|
|     /sts-71/launch/|
|/history/apollo/a...|
|/history/apollo/a...|
|/shuttle/resource...|
|/www/software/win...|
|/pub/winvn/releas...|
|/history/apollo/a...|
|/history/apollo/a...|
|/history/history.htm|
+--------------------+



#### Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan 

In [95]:
endpointstop25 = sqlContext.sql("""
    SELECT endpoint, COUNT(code) AS codigo404 
    FROM nasa
    WHERE code = '404'
    GROUP BY endpoint
    ORDER BY COUNT(code) DESC
    LIMIT 25
""")
endpointstop25.show()

+--------------------+---------+
|            endpoint|codigo404|
+--------------------+---------+
|/pub/winvn/releas...|        4|
|/history/apollo/a...|        4|
|/history/apollo/a...|        2|
|/history/apollo/a...|        2|
|/pub/winvn/readme...|        2|
|/elv/DELTA/uncons...|        1|
|     /sts-71/launch/|        1|
|/history/apollo/a...|        1|
|/shuttle/resource...|        1|
|/history/apollo/a...|        1|
|/history/apollo/a...|        1|
|/www/software/win...|        1|
|/history/history.htm|        1|
+--------------------+---------+



#### El top 5 de días que se generaron código de respuestas 404 

In [96]:
top5dias = sqlContext.sql("""
    SELECT date, COUNT(code) AS TotalRespuestas404
    FROM nasa
    WHERE code = '404'
    GROUP BY date
    ORDER BY COUNT(code) DESC
    LIMIT 5
""")
top5dias.show()

+-----------+------------------+
|       date|TotalRespuestas404|
+-----------+------------------+
|01/Aug/1995|                22|
+-----------+------------------+



In [97]:
sc.stop()